In [31]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, KFold, train_test_split, cross_val_score, cross_val_predict, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from scipy.stats import pearsonr, sem

In [2]:
def get_X_y(conn_mat_filename: str, general_scores_filename: str):
    Glasser_conn_mat = np.load(conn_mat_filename)

    # get upper triangle indices, without diagonal 0 (k = 1 starts from the k' diagonal)
    indices = np.triu_indices(360, k=1)

    # create X: rows for subjects, and flattened upper triangle mat for each subject
    X = []
    for i in range(Glasser_conn_mat.shape[2]):
        X.append(Glasser_conn_mat[:, :, i][indices])
    # convert X to data frame for pipeline parameters
    X = pd.DataFrame(X)

    # read scores
    y = pd.read_csv(general_scores_filename, header=None).to_numpy()

    return X, y

In [3]:
X, y = get_X_y("data/Glasser_conn_mat_322_subj.npy", "data/general_scores_322_subj.csv")

In [4]:
X

,0,1,2,3,4,5,6,7,8,9,...,64610,64611,64612,64613,64614,64615,64616,64617,64618,64619
0,0.298940,0.732744,0.918536,0.839563,0.432511,0.442482,0.264298,0.397874,0.363293,0.242166,...,0.350374,-0.091814,-0.072425,0.042511,-0.128833,-0.028425,-0.054679,0.021419,-0.038918,0.255055
1,0.012735,0.639222,0.825878,0.742386,0.632681,0.491353,0.072223,0.309493,0.335780,0.303365,...,0.235892,0.060626,0.244300,0.238003,0.008831,0.105045,0.147294,0.118197,0.103300,0.386892
2,0.504869,0.793886,0.670200,0.736328,0.275567,0.418920,0.662700,0.645710,0.561706,0.607769,...,0.189165,0.301933,0.253142,0.173838,0.151999,0.490085,0.364271,0.171950,0.134088,0.641098
3,0.434921,0.625641,0.866148,0.796666,0.615978,0.449730,0.458218,0.470296,0.479121,0.247320,...,0.476037,0.116935,0.211443,0.206184,0.161278,0.283144,0.201882,0.168905,0.232736,0.466078
4,0.329170,0.684801,0.879460,0.848767,0.750146,0.717157,0.090091,0.039740,0.348103,0.159279,...,0.473147,0.021511,0.135669,0.235904,0.133697,0.182638,0.183275,0.171366,0.136573,0.284417
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
317,0.614404,0.776613,0.931250,0.912197,0.827961,0.740271,0.632516,0.538272,0.654627,0.579273,...,0.221746,0.028976,0.111459,0.018839,0.042003,-0.053316,-0.165810,0.195446,0.243251,0.484157
318,0.206464,0.591203,0.831999,0.796237,0.584414,0.506734,0.296378,0.323051,0.333396,0.076793,...,0.570339,0.149866,0.088419,0.147588,0.335105,0.178705,0.190511,0.100918,0.140988,0.321525
319,0.144381,0.508423,0.848602,0.766589,0.585104,0.261164,0.107827,0.285483,0.113639,0.034838,...,0.573053,0.367515,0.210395,0.039314,0.256437,0.520648,0.165135,0.057304,0.048039,0.474967
320,0.237676,0.520094,0.939677,0.912065,0.806352,0.690207,0.069966,-0.150622,0.494487,0.408926,...,0.354727,0.017433,-0.069727,-0.020538,-0.132383,0.055792,0.062163,0.197313,0.133259,0.697519


In [5]:
y

array([[613.],
       [724.],
       [703.],
       [668.],
       [690.],
       [698.],
       [711.],
       [759.],
       [513.],
       [728.],
       [630.],
       [596.],
       [678.],
       [690.],
       [610.],
       [623.],
       [530.],
       [713.],
       [517.],
       [635.],
       [627.],
       [630.],
       [615.],
       [664.],
       [620.],
       [776.],
       [741.],
       [692.],
       [663.],
       [620.],
       [685.],
       [698.],
       [770.],
       [687.],
       [522.],
       [620.],
       [690.],
       [729.],
       [690.],
       [732.],
       [604.],
       [721.],
       [721.],
       [721.],
       [550.],
       [750.],
       [600.],
       [730.],
       [667.],
       [595.],
       [697.],
       [672.],
       [699.],
       [718.],
       [728.],
       [581.],
       [658.],
       [689.],
       [650.],
       [641.],
       [660.],
       [729.],
       [678.],
       [625.],
       [726.],
       [742.],
       [65

In [6]:
print(f"Mean General Score: {np.round(np.mean(y), 2)}")
print(f"Standard deviation: {np.round(np.std(y), 2)}")

Mean General Score: 658.28
Standard deviation: 64.65


# Linear Regression

In [35]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = LinearRegression()

# Report cross validation score across all folds
cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation negative MSE score: ", np.round(cross_val_score_list, 2))
print("mean MSE across folds:", np.round(-1 * np.mean(cross_val_score_list), 2))
print("MSE standard error across folds:", np.round(sem(cross_val_score_list), 2))

# Fit the model to train data 
lr.fit(X_train_pca, y_train)

# Predict on unseen test set
y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation negative MSE score:  [-1.13 -1.22 -0.77 -1.5  -0.97 -1.18 -0.9  -0.91 -1.53 -1.47]
mean MSE across folds: 1.16
MSE standard error across folds: 0.09
Test MSE: 0.589, r: 0.309, p value: 0.08


# Ridge

In [36]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

rdg = Ridge()

# Report cross validation score across all folds
cross_val_score_list = cross_val_score(rdg, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation negative MSE score: ", np.round(cross_val_score_list, 2))
print("mean MSE across folds:", np.round(-1 * np.mean(cross_val_score_list), 2))
print("MSE standard error across folds:", np.round(sem(cross_val_score_list), 2))

# Fit the model to train data 
rdg.fit(X_train_pca, y_train)

# Predict on unseen test set
y_test_predicted = rdg.predict(X_test_pca)
test_mse_rdg = mean_squared_error(y_test, y_test_predicted)
r_rdg, p_val_rdg = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_rdg, 3)}, r: {np.round(r_rdg, 3)}, p value: {np.round(p_val_rdg, 3)}")

cross validation negative MSE score:  [-1.13 -1.22 -0.77 -1.5  -0.97 -1.18 -0.9  -0.91 -1.53 -1.47]
mean MSE across folds: 1.16
MSE standard error across folds: 0.09
Test MSE: 0.589, r: 0.309, p value: 0.08


# Lasso

In [37]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = Lasso()

# Report cross validation score across all folds
cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation negative MSE score: ", np.round(cross_val_score_list, 2))
print("mean MSE across folds:", np.round(-1 * np.mean(cross_val_score_list), 2))
print("MSE standard error across folds:", np.round(sem(cross_val_score_list), 2))

# Fit the model to train data
lr.fit(X_train_pca, y_train)

# Predict on unseen test set
y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation negative MSE score:  [-0.98 -1.2  -0.75 -1.39 -0.9  -0.99 -0.87 -0.9  -1.11 -1.43]
mean MSE across folds: 1.05
MSE standard error across folds: 0.07
Test MSE: 0.626, r: 0.224, p value: 0.21


# Elastic net

In [38]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

lr = ElasticNet()

# Report cross validation score across all folds
cross_val_score_list = cross_val_score(lr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation negative MSE score: ", np.round(cross_val_score_list, 2))
print("mean MSE across folds:", np.round(-1 * np.mean(cross_val_score_list), 2))
print("MSE standard error across folds:", np.round(sem(cross_val_score_list), 2))

# Fit the model to train data
lr.fit(X_train_pca, y_train)

# Predict on unseen test set
y_test_predicted = lr.predict(X_test_pca)
test_mse_lr = mean_squared_error(y_test, y_test_predicted)
r_lr, p_val_lr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_lr, 3)}, r: {np.round(r_lr, 3)}, p value: {np.round(p_val_lr, 3)}")

cross validation negative MSE score:  [-1.04 -1.21 -0.76 -1.44 -0.93 -1.07 -0.87 -0.9  -1.23 -1.43]
mean MSE across folds: 1.09
MSE standard error across folds: 0.07
Test MSE: 0.604, r: 0.287, p value: 0.105


# Random Forest

In [42]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=75, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

# Parameters grid for hyperparameter tuning 
param_grid_rf = {
    'n_estimators': [50, 100, 200],  # Number of trees in the forest
    'max_depth': [None, 5, 10],  # Maximum depth of each tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required to be at a leaf node
    'max_features': ['sqrt', 'log2'],  # Number of features to consider when looking for the best split
}

r_vec = []
p_value_vec = []
fold_mse_train = []
fold_mse_val = []
fold_config = []

kf = KFold(n_splits=10, shuffle=True, random_state=0)

for train_index, val_index in kf.split(X_train_pca):
    X_train_fold, X_val_fold = X_train_pca[train_index], X_train_pca[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

    random_forest = RandomForestRegressor(random_state=0)
    grid_search = GridSearchCV(
        random_forest,
        param_grid=param_grid_rf,
        scoring="neg_mean_squared_error",
        cv=10,
        n_jobs=-1,
    )
    grid_search.fit(X_train_fold, y_train_fold)

    # Fit and Predict
    best_estimator = grid_search.best_estimator_
    y_train_predicted = best_estimator.predict(X_train_fold)
    y_predicted = best_estimator.predict(X_val_fold)
    fold_config.append(grid_search.best_params_)
    train_mse, val_mse = mean_squared_error(y_train_fold, y_train_predicted), mean_squared_error(y_val_fold, y_predicted)

    r, p_value = pearsonr(y_predicted, y_val_fold)
    r_vec.append(r)
    p_value_vec.append(p_value)
    fold_mse_train.append(train_mse)
    fold_mse_val.append(val_mse)

print("mean r: ", np.round(np.mean(r_vec), 2), ", mean p-value: ", np.round(np.mean(p_value_vec), 2))

print(f"min MSE: {np.round(np.min(fold_mse_val), 2)} in fold {np.argmin(fold_mse_val) + 1}")
# Configuration that received the minimal MSE
config = fold_config[np.argmin(fold_mse_val)]
print(config)

# Random Forest Regressor with the most accurate configuration
chosen_random_forest = RandomForestRegressor(random_state=0, **config)

# Report cross validation score across all folds
cross_val_score_list = cross_val_score(chosen_random_forest, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation negative MSE score: ", np.round(cross_val_score_list, 2))
print("mean MSE across folds:", np.round(-1 * np.mean(cross_val_score_list), 2))
print("MSE standard error across folds:", np.round(sem(cross_val_score_list), 2))

# Fit the model to train data
chosen_random_forest.fit(X_train_pca, y_train)

# Predict on unseen test set
y_pred = chosen_random_forest.predict(X_test_pca)
mse = mean_squared_error(y_test, y_pred)
r, p_value = pearsonr(y_test, y_pred)
print(f"MSE: {np.round(mse, 3)}, r: {np.round(r, 3)}, p value: {np.round(p_value, 3)}")


mean r:  0.15 , mean p-value:  0.39
min MSE: 0.69 in fold 6
{'max_depth': None, 'max_features': 'log2', 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 50}
cross validation negative MSE score:  [-0.79 -1.11 -0.71 -1.27 -0.79 -0.82 -0.87 -0.98 -0.88 -1.29]
mean MSE across folds: 0.95
MSE standard error across folds: 0.07
MSE: 0.551, r: 0.298, p value: 0.092


# Gradient Boosting

In [43]:
# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

# Normalization of features and behavioral scores
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

y_train = scaler.fit_transform(y_train).ravel()
y_test = scaler.transform(y_test).ravel()

# Apply PCA for feature selection
pca = PCA(n_components=20, svd_solver="full", random_state=0)
X_train_pca = pca.fit_transform(X_train)
# Apply PCA on the testing data
X_test_pca = pca.transform(X_test)

gbr = GradientBoostingRegressor(random_state=0)

# Report cross validation score across all folds
cross_val_score_list = cross_val_score(gbr, X_train_pca, y_train, cv=10, scoring="neg_mean_squared_error")
print("cross validation negative MSE score: ", np.round(cross_val_score_list, 2))
print("mean MSE across folds:", np.round(-1 * np.mean(cross_val_score_list), 2))
print("MSE standard error across folds:", np.round(sem(cross_val_score_list), 2))

# Fit the model to train data
gbr.fit(X_train_pca, y_train)

# Predict on unseen test set
y_test_predicted = gbr.predict(X_test_pca)
test_mse_gbr = mean_squared_error(y_test, y_test_predicted)
r_gbr, p_val_gbr = pearsonr(y_test, y_test_predicted)
print(f"Test MSE: {np.round(test_mse_gbr, 3)}, r: {np.round(r_gbr, 3)}, p value: {np.round(p_val_gbr, 3)}")

cross validation negative MSE score:  [-0.89 -1.41 -0.84 -1.69 -1.19 -1.48 -1.17 -0.88 -1.2  -1.45]
mean MSE across folds: 1.22
MSE standard error across folds: 0.09
Test MSE: 0.503, r: 0.37, p value: 0.034
